## Import needed packages

In [ ]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
from cellpose import models, io, utils
from tqdm import tqdm


## Create Functions
Using two functions for this analysis. 

`read_and_prep_images` reads in the czi scenes as NumPy compatible arrays, removes the empty dimensions and then reorders the channel axis to be the last axis for compatibility with scikit image. Finally, the nuclei channel is copied and a gaussian blur is applied for improved nuclei segmentation with CellPose.

`segment_and_measure_CP` runs the gaussian blurred nuclei images through CellPose with the nuclei model. The masks are then used with the reordered images to collect fluorescence intensity measurements. The gaussian blurred nuclei channel, masks, reordered images, and measurements are all saved.

In [ ]:
def read_and_prep_images(czifile):
    scenes = [czifile.read_image(S=s)[0] for s in range(czifile.size[2])]
    squeezed = [np.squeeze(i) for i in scenes]
    reorder = [np.moveaxis(i,0,-1) for i in squeezed] #channel axis needs to be placed last for scikit-image
    nuclei_channel = [reorder[i][...,-1] for i in range(len(reorder))]
    gauss_nuc = [sk.filters.gaussian(img, sigma=3) for img in nuclei_channel]
    return scenes, reorder, gauss_nuc, nuclei_channel        


In [ ]:
def segment_and_measure_CP(reorder, gauss_nuc, props, save_path, img_name):
    for i in range(len(gauss_nuc)):
        model = models.Cellpose(gpu=True,model_type='nuclei') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
        channels =[[0,0]]
        masks, _, _, _ = model.eval(gauss_nuc[i], diameter=150, channels=channels, z_axis=0,stitch_threshold=0.1,flow_threshold=0.4,cellprob_threshold=0.0, min_size=100)
        # filtered_masks = sk.segmentation.clear_border(masks) #nuclei span the whole image in Z and are removed even if they are complete objects
        df = sk.measure.regionprops_table(masks,reorder[i],properties=props)
        df = pd.DataFrame.from_dict(df)
        #Save Everything
        dataframe_path = os.path.join(save_path,'measurements')
        df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'_scene0'+str(i+1)+'.csv'))
        masks_path = os.path.join(save_path,'masks')
        sk.io.imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'_scene0'+str(i+1)+'.tif'),masks,check_contrast=False)
        scenes_path = os.path.join(save_path,'scenes')
        sk.io.imsave(os.path.join(scenes_path, img_name[:-4]+'_scene0'+str(i+1)+'.tif'),reorder[i])

## Running the pipeline

Need to give the functions the location of the flies and save directory, and read in the images with `CziFile`. Then process all images through the above functions with a for loop. Within the loop, use `os.path.basename` to get the file name for saving the outputs and also provide the properties to measure with `sk.measure.regionprops_table`.

In [ ]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))
save_path = "E:/Fondufe-Mittendorf_Lab/OIC-150_SATB2-GFP_circ3915-3XFLAG_Fluorescence_Quantification/"
for i in tqdm(range(len(czi_files))):
    img_name = os.path.basename(file_paths[i])
    scenes, reorder, gauss_nuc, nuclei_channel = read_and_prep_images(czi_files[i])
    props = ('label', 'intensity_mean')
    segment_and_measure_CP(reorder, gauss_nuc, props,save_path,img_name)